<a href="https://colab.research.google.com/github/Papa-Panda/Paper_reading/blob/main/DPO(unfinished).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# DPO 原理 https://blog.csdn.net/qq_27590277/article/details/142383849

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Let's assume you have a base language model (transformer-based)
class LanguageModel(nn.Module):
    def __init__(self, transformer_model):
        super(LanguageModel, self).__init__()
        self.transformer = transformer_model  # Pre-trained transformer model like GPT
        self.classifier = nn.Linear(transformer_model.config.hidden_size, 1)  # Output a score for each response

    def forward(self, input_ids, attention_mask=None):
        # Forward pass through the transformer
        transformer_output = self.transformer(input_ids, attention_mask=attention_mask).last_hidden_state

        # Take the hidden state of the [CLS] token (for simplicity)
        cls_embedding = transformer_output[:, 0, :]

        # Output a single score for the response
        score = self.classifier(cls_embedding)
        return score

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Let's assume you have a base language model (transformer-based)
class LanguageModel(nn.Module):
    def __init__(self, transformer_model):
        super(LanguageModel, self).__init__()
        self.transformer = transformer_model  # Pre-trained transformer model like GPT
        self.classifier = nn.Linear(transformer_model.config.hidden_size, 1)  # Output a score for each response

    def forward(self, input_ids, attention_mask=None):
        # Forward pass through the transformer
        transformer_output = self.transformer(input_ids, attention_mask=attention_mask).last_hidden_state

        # Take the hidden state of the [CLS] token (for simplicity)
        cls_embedding = transformer_output[:, 0, :]

        # Output a single score for the response
        score = self.classifier(cls_embedding)
        return score


# Assume we have a pre-trained transformer model
pretrained_transformer = ...  # Load a transformer like GPT or BERT
model = LanguageModel(pretrained_transformer).cuda()

# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

# Loss function: Binary cross-entropy for pairwise comparisons
def dpo_loss(preferred_score, non_preferred_score):
    return -torch.log(torch.sigmoid(preferred_score - non_preferred_score))

# Example training loop
def train_dpo(model, dataloader, optimizer):
    model.train()

    for batch in dataloader:
        # Assume the batch consists of input pairs and their preference labels
        preferred_inputs = batch['preferred_input_ids'].cuda()  # Preferred response input IDs
        non_preferred_inputs = batch['non_preferred_input_ids'].cuda()  # Non-preferred response input IDs

        preferred_attention_mask = batch['preferred_attention_mask'].cuda()
        non_preferred_attention_mask = batch['non_preferred_attention_mask'].cuda()

        # Forward pass: Compute scores for both preferred and non-preferred responses
        preferred_score = model(preferred_inputs, attention_mask=preferred_attention_mask)
        non_preferred_score = model(non_preferred_inputs, attention_mask=non_preferred_attention_mask)

        # Compute DPO loss
        loss = dpo_loss(preferred_score, non_preferred_score)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Optionally, log loss and other metrics
        print(f"Training loss: {loss.item()}")

# Dataset for DPO (pseudo code)
# You need to prepare a dataset with pairs of responses
dataloader = ...  # Assume we have a dataloader that loads pairs of responses

# Training
for epoch in range(10):
    train_dpo(model, dataloader, optimizer)
